In [60]:
# 필요 패키지 로드
# RPA
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
# Crawling
import pandas as pd
import requests
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm # 반복문 실행시 실행도 확인용 라이브러리

In [61]:
# 한국무역협회 사이트 접속
URL = "https://www.kita.net/cmmrcInfo/ehgtGnrlzInfo/rltmEhgt.do"
driver = webdriver.Chrome('C:\\Workspace\\python\\Toy_Project_2022\\MBTI\\chrome\\chromedriver.exe')
driver.get(URL)
time.sleep(2) # 로딩시간에 맞게 적정히 수정

<ipython-input-61-284dca088280>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\\Workspace\\python\\Toy_Project_2022\\MBTI\\chrome\\chromedriver.exe')


In [62]:
# 통화별 환율관련 수치 가공 후 리스트로 저장
import re

data_01 = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[3]/div[2]').text
# data_01
data_02 = re.sub('[^0-9a-zA-Z.\n ]', '', data_01) 
data_03 = data_02.replace(',', '')
data_04 = data_03.replace(',', '')
data_05 = data_04.split("\n")[1:]
data_05

['USD  1285.00  6.50 0.50 1307.48 1262.52 1297.40 1272.60',
 'JPY  950.80  4.03 0.42 967.43 934.17 960.02 941.58',
 'EUR  1359.40  2.49 0.18 1386.31 1332.49 1372.99 1345.81',
 'CNY  192.18  1.20 0.62 203.71 182.58 194.10 190.26',
 'HKD  163.76  0.78 0.47 167.01 160.51 165.39 162.13',
 'GBP  1578.37  8.63 0.54 1609.77 1546.97 1594.15 1562.59',
 'CHF  1341.20  10.73 0.79 1367.88 1314.52 1354.61 1327.79',
 'CAD  996.74  2.64 0.26 1016.57 976.91 1006.70 986.78',
 'AUD  890.76  4.77 0.53 908.48 873.04 899.66 881.86',
 'NZD  811.35  3.72 0.46 827.49 795.21 819.46 803.24',
 'SEK  127.73  0.50 0.39 130.92 124.54 129.00 126.46',
 'DKK  182.75  0.50 0.39 187.31 178.19 184.57 180.93',
 'NOK  130.99  1.31 1.01 134.26 127.72 132.29 129.69',
 'SAR  342.40  1.77 0.51 359.52 318.44 345.82 338.98',
 'KWD  4192.50  22.58 0.54 4444.05 3857.10 4234.42 4150.58',
 'BHD  3408.49  17.24 0.50 3612.99 3135.82 3442.57 3374.41',
 'AED   349.86  1.75 0.50 367.35 325.37 353.35 346.37',
 'THB  36.36  0.01 0.03 38.52

In [63]:
data_05[0].split()

['USD', '1285.00', '6.50', '0.50', '1307.48', '1262.52', '1297.40', '1272.60']

In [64]:
# 날짜 데이터 추출 후 리스트에 날짜 추가
date = driver.find_element(By.XPATH, f'//*[@id="contents"]/div[3]/div[1]/div[1]').text
date_01 = date.split('\n')
date_02 = date_01[0].replace('년', '-')
date_03 = date_02.replace('월', '-')
date_04 = date_03.replace('일', '')
[date_04] + data_05[0].split()

['2022-06-27',
 'USD',
 '1285.00',
 '6.50',
 '0.50',
 '1307.48',
 '1262.52',
 '1297.40',
 '1272.60']

In [65]:
# 데이터 프레임에 넣기
df_01 = pd.DataFrame({'Date' : [], 'Currency' : [], 
              'Exchange_rate' : [], 'Day_to_day' :[],
              'Rise_and_fall' : [],
              'Buy' : [], 'Sell' : [],
              'To_Transfer' : [], 'Receive' : []})
for i in range(len(data_05)):
    df_01.loc[len(df_01)] = [date_04] + data_05[i].split()

df_01

,Date,Currency,Exchange_rate,Day_to_day,Rise_and_fall,Buy,Sell,To_Transfer,Receive
0,2022-06-27,USD,1285.00,6.50,0.50,1307.48,1262.52,1297.40,1272.60
1,2022-06-27,JPY,950.80,4.03,0.42,967.43,934.17,960.02,941.58
2,2022-06-27,EUR,1359.40,2.49,0.18,1386.31,1332.49,1372.99,1345.81
3,2022-06-27,CNY,192.18,1.20,0.62,203.71,182.58,194.10,190.26
4,2022-06-27,HKD,163.76,0.78,0.47,167.01,160.51,165.39,162.13
5,2022-06-27,GBP,1578.37,8.63,0.54,1609.77,1546.97,1594.15,1562.59
6,2022-06-27,CHF,1341.20,10.73,0.79,1367.88,1314.52,1354.61,1327.79
7,2022-06-27,CAD,996.74,2.64,0.26,1016.57,976.91,1006.70,986.78
8,2022-06-27,AUD,890.76,4.77,0.53,908.48,873.04,899.66,881.86
9,2022-06-27,NZD,811.35,3.72,0.46,827.49,795.21,819.46,803.24


In [66]:
# https://docs.microsoft.com/ko-kr/azure/azure-sql/database/connect-query-python?view=azuresql
# 해당 정보들은 반드시 메모장이나 다른 매체로 기록해서 입력할 것
# sql 접속 
# import pyodbc
# server = '<server>.database.windows.net'
# database = '<database>'
# username = '<username>'
# password = '{<password>}'   
# driver= '{ODBC Driver 17 for SQL Server}'

# with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
#     with conn.cursor() as cursor:
#         cursor.execute("SELECT TOP 3 name, collation_name FROM sys.databases")
#         row = cursor.fetchone()
#         while row:
#             print (str(row[0]) + " " + str(row[1]))
#             row = cursor.fetchone()


# sql 접속 정보
info = pd.read_csv("C:\Workspace\python\Toy_Project_2022\ADF_ETL\info\sqlinfo.txt", sep=":")

import pyodbc
server = info.loc[0][0] + '.database.windows.net' # sql 서버 이름
database = info.loc[0][3] # sql db이름
username = info.loc[0][1] # sql 서버 관리자 ID
password = info.loc[0][2] # sql 서버 관리자 PW
driver= '{ODBC Driver 17 for SQL Server}'
# 'DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password
with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("SELECT TOP 3 name, collation_name FROM sys.databases")
        row = cursor.fetchone()
        while row:
            print (str(row[0]) + " " + str(row[1]))
            row = cursor.fetchone()

master SQL_Latin1_General_CP1_CI_AS
sql_db_tt_jb1519 SQL_Latin1_General_CP1_CI_AS


In [67]:
# 데이터 적재
import pyodbc
import pandas as pd
# insert data from csv file into dataframe.
# working directory for csv file: type "pwd" in Azure Data Studio or Linux
# working directory in Windows c:\users\username

# df = pd.read_csv("c:\\user\\username\department.csv")

# Some other example server values are
# server = 'localhost\sqlexpress' # for a named instance
# server = 'myserver,port' # to specify an alternate port
server = info.loc[0][0] + '.database.windows.net' # sql 서버 이름
database = info.loc[0][3] # sql db이름
username = info.loc[0][1] # sql 서버 관리자 ID
password = info.loc[0][2] # sql 서버 관리자 PW
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()
# Insert Dataframe into SQL Server:
for index, row in df_01.iterrows():
     cursor.execute("INSERT INTO dbo.Exchange_rate_test (Date,Currency,Exchange_rate,Day_to_day,Rise_and_fall,Buy,Sell,To_Transfer,Receive) values(?,?,?,?,?,?,?,?,?)", 
                     row.Date, row.Currency, row.Exchange_rate, row.Day_to_day, row.Rise_and_fall, row.Buy, row.Sell, row.To_Transfer, row.Receive)
cnxn.commit()
cursor.close()

In [68]:
for index, row in df_01.iterrows():
    print(index, row, sep='\n')

0
Date             2022-06-27
Currency                USD
Exchange_rate       1285.00
Day_to_day             6.50
Rise_and_fall          0.50
Buy                 1307.48
Sell                1262.52
To_Transfer         1297.40
Receive             1272.60
Name: 0, dtype: object
1
Date             2022-06-27
Currency                JPY
Exchange_rate        950.80
Day_to_day             4.03
Rise_and_fall          0.42
Buy                  967.43
Sell                 934.17
To_Transfer          960.02
Receive              941.58
Name: 1, dtype: object
2
Date             2022-06-27
Currency                EUR
Exchange_rate       1359.40
Day_to_day             2.49
Rise_and_fall          0.18
Buy                 1386.31
Sell                1332.49
To_Transfer         1372.99
Receive             1345.81
Name: 2, dtype: object
3
Date             2022-06-27
Currency                CNY
Exchange_rate        192.18
Day_to_day             1.20
Rise_and_fall          0.62
Buy                  203.71